Jacqueline Sanchez <br> 
June 21, 2024 <br>
COSC 3337 | Summer 2024

## Lab4

### 1. State the difference and similarity of GD and SGD, give each two situitions, one should use GD, the other should use SGD. Explain why. (10)

<span style = 'color:green'>
Gradient Descent processes the entire dataset for each update, leading to stable but slower convergence, making it suitable for small datasets and convex loss functions. Stochastic Gradient Descent, on the other hand, updates parameters more frequently using individual data points which speeds up convergence but introduces more noise. Both methods aim to minimize loss functions by adjusting model parameters based on gradient calculations. Since we are dealing with a smaller dataset, it is preferable to use Gradient Descent for this case.

In [7]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score

In [2]:
# STAB = pd.read_csv("smart_grid_stability_augmented.csv") # just to observe the data before processing it
# STAB.head(5)

In [8]:
STAB = pd.read_csv("smart_grid_stability_augmented.csv")
STAB = STAB.drop('stab', axis = 1)
STAB.head(5) # seems like the response variable is 'stabf'

tau1      tau2      tau3      tau4        p1        p2        p3  \
0  2.959060  3.079885  8.381025  9.780754  3.763085 -0.782604 -1.257395   
1  9.304097  4.902524  3.047541  1.369357  5.067812 -1.940058 -1.872742   
2  8.971707  8.848428  3.046479  1.214518  3.405158 -1.207456 -1.277210   
3  0.716415  7.669600  4.486641  2.340563  3.963791 -1.027473 -1.938944   
4  3.134112  7.608772  4.943759  9.857573  3.525811 -1.125531 -1.845975   

         p4        g1        g2        g3        g4     stabf  
0 -1.723086  0.650456  0.859578  0.887445  0.958034  unstable  
1 -1.255012  0.413441  0.862414  0.562139  0.781760    stable  
2 -0.920492  0.163041  0.766689  0.839444  0.109853  unstable  
3 -0.997374  0.446209  0.976744  0.929381  0.362718  unstable  
4 -0.554305  0.797110  0.455450  0.656947  0.820923  unstable

In [9]:
num_rows, num_cols = STAB.shape
print(num_rows, num_cols)

60000 13


### 2. Seperate the STAB dataset to train and test, stabf is y, the others are x, make test dataset 30% of the total dataset (10)

In [10]:
#Write your code here
X = STAB.drop('stabf', axis = 1)
y = STAB['stabf']

X_train, X_test,y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

# print(X_train.head())
# print(y_train.head())                                                

### 3. Apply Random Forest to this dataset, predict stabf, show the accuracy. (10)

In [11]:
#Write your code here

# 1. train the Random Forest model
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# 2. make predictions
y_pred = rf.predict(X_test)

# 3. evaluate the model
accuracy = accuracy_score(y_test, y_pred)


print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.95


### 4. Apply SVM to this dataset, predict stabf, show the accuracy. (10)

In [12]:
wine = pd.read_csv("Wine.csv")
wine.head(5)

Alcohol  Malic_Acid   Ash  Ash_Alcanity  Magnesium  Total_Phenols  \
0    14.23        1.71  2.43          15.6        127           2.80   
1    13.20        1.78  2.14          11.2        100           2.65   
2    13.16        2.36  2.67          18.6        101           2.80   
3    14.37        1.95  2.50          16.8        113           3.85   
4    13.24        2.59  2.87          21.0        118           2.80   

   Flavanoids  Nonflavanoid_Phenols  Proanthocyanins  Color_Intensity   Hue  \
0        3.06                  0.28             2.29             5.64  1.04   
1        2.76                  0.26             1.28             4.38  1.05   
2        3.24                  0.30             2.81             5.68  1.03   
3        3.49                  0.24             2.18             7.80  0.86   
4        2.69                  0.39             1.82             4.32  1.04   

   OD280  Proline  Customer_Segment  
0   3.92     1065                 1  
1   3.40     1050                 1  
2   3.17     1185                 1  
3   3.45     1480                 1  
4   2.93      735                 1

In [14]:
# wine.isnull().sum()

In [39]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Load datasets
STAB = pd.read_csv("smart_grid_stability_augmented.csv")
wine = pd.read_csv("wine.csv")  # Ensure you have loaded the wine dataset as well

# Concatenate datasets by aligning columns
combined_df = pd.concat([STAB, wine], axis=0, ignore_index=True, sort=False)

# Print combined dataset to check
print(combined_df)

# Handle missing values
# Remove columns with more than 50% missing values
threshold = len(combined_df) * 0.5
combined_df = combined_df.dropna(thresh=threshold, axis=1)

# Separate numeric and non-numeric columns
numeric_cols = combined_df.select_dtypes(include=['number']).columns
non_numeric_cols = combined_df.select_dtypes(exclude=['number']).columns

# Impute missing values in numeric columns with the mean
combined_df[numeric_cols] = combined_df[numeric_cols].fillna(combined_df[numeric_cols].mean())

# Impute missing values in non-numeric columns with the mode
for col in non_numeric_cols:
    combined_df[col] = combined_df[col].fillna(combined_df[col].mode()[0])

# Print the dataframe after handling missing values to verify
print(combined_df.isnull().sum())

# Split target from the rest of the features
X = combined_df.drop('stabf', axis=1)  # Replace 'stabf' with your response variable name if different
y = combined_df['stabf']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preparing for svm model training

# 1. Train the svm model
svm = SVC(kernel='linear', random_state=42)
svm.fit(X_train, y_train)

# 2. Make predictions
y_pred_svm = svm.predict(X_test)

# 3. Evaluate the model
accuracy_svm = accuracy_score(y_test, y_pred_svm)

print(f'SVM Accuracy: {accuracy_svm:.2f}')


# Load datasets
STAB = pd.read_csv("smart_grid_stability_augmented.csv")
wine = pd.read_csv("wine.csv")  # Ensure you have loaded the wine dataset as well

# concatinating datasets by aligning columns
combined_df = pd.concat([STAB, wine], axis=0, ignore_index=True, sort=False)
# print(combined_df)


# handling missing values
threshold = len(combined_df) * 0.6 # remove columns with more than 60% missing values
combined_df = combined_df.dropna(thresh=threshold, axis=1)

# separate numeric and non-numeric columns
numeric_cols = combined_df.select_dtypes(include=['number']).columns
non_numeric_cols = combined_df.select_dtypes(exclude=['number']).columns

# impute missing values in numeric columns with the mean
combined_df[numeric_cols] = combined_df[numeric_cols].fillna(combined_df[numeric_cols].mean())

# impute missing values in non-numeric columns with the mode
for col in non_numeric_cols:
    combined_df[col] = combined_df[col].fillna(combined_df[col].mode()[0])

# print(combined_df.isnull().sum())



########### Performing SVM ############

# Split target from the rest of the features
X = combined_df.drop('stabf', axis=1)  # Replace 'stabf' with your response variable name if different
y = combined_df['stabf']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preparing for svm model training

# 1. Train the svm model
svm = SVC(kernel='linear', random_state=42)
svm.fit(X_train, y_train)

# 2. Make predictions
y_pred_svm = svm.predict(X_test)

# 3. Evaluate the model
accuracy_svm = accuracy_score(y_test, y_pred_svm)

print(f'SVM Accuracy: {accuracy_svm:.2f}')


           tau1      tau2      tau3      tau4        p1        p2        p3  \
0      2.959060  3.079885  8.381025  9.780754  3.763085 -0.782604 -1.257395   
1      9.304097  4.902524  3.047541  1.369357  5.067812 -1.940058 -1.872742   
2      8.971707  8.848428  3.046479  1.214518  3.405158 -1.207456 -1.277210   
3      0.716415  7.669600  4.486641  2.340563  3.963791 -1.027473 -1.938944   
4      3.134112  7.608772  4.943759  9.857573  3.525811 -1.125531 -1.845975   
...         ...       ...       ...       ...       ...       ...       ...   
60173       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
60174       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
60175       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
60176       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
60177       NaN       NaN       NaN       NaN       NaN       NaN       NaN   

             p4        g1        g2  ...  Magnesium

In [9]:
X=wine.drop("Customer_Segment",axis=1).values
y=wine["Customer_Segment"].values

### 5. Standartize the variables X (10)

In [10]:
#Write your code here.
from sklearn.preprocessing import StandardScaler

# 1. invoke scaler
scaler = StandardScaler()

# 2. fit and transform the training data
X_train_scaled = scaler.fit_transform(X_train)

# 3. transform the test data 
X_test_scaled = scaler.transform(X_test)

# viewing the standardized data
print(pd.DataFrame(X_train_scaled, columns = X_train.columns).head())

       tau1      tau2      tau3      tau4        p1        p2        p3  \
0  0.944087  1.322404  0.212504  0.236362  0.308582 -0.848871  1.201085   
1 -0.819194  1.252164  0.296433 -1.599900  0.635702  1.104413 -1.583831   
2  0.997219 -0.107312  0.970398  0.921308 -0.245331  1.314141 -0.051993   
3 -1.097213  0.708523  1.105498  0.934111 -1.754412  1.697428  1.352325   
4 -0.285885  0.964390  0.784895 -1.501700 -0.703486 -1.616879  1.500220   

         p4        g1        g2        g3        g4  
0 -0.892750  1.485538  0.208525  0.421753 -0.798097  
1 -0.621148  1.454570  1.522550  0.979838 -1.426316  
2 -0.833814  1.519562  1.073471  1.634525 -0.450236  
3  0.000293 -0.590446 -1.416779 -1.615071  0.908748  
4  1.334720 -1.394408  1.080738  0.022626  1.046577  


### 6. Use PCA to reduce standartized X to 2 dimensions (10)

In [11]:
#Write your code here.
from sklearn.decomposition import PCA

# 1. initialize PCA with the number of components (in this case 2 dim)
pca = PCA(n_components = 2)

# 2. fit and transform the training data
X_train_pca = pca.fit_transform(X_train_scaled)

# 3. transform the test data
x_test_pca = pca.transform(X_test_scaled)

print(pd.DataFrame(X_train_pca, columns = ['PC1', 'PC2']).head())

        PC1       PC2
0 -0.472485 -1.522133
1 -0.870298 -1.093831
2  0.316512 -1.880438
3  2.509509  1.677581
4  1.014759  0.467995


### 7. Use LDA to reduce standartized X to 2 dimensions (10)

In [12]:
#Write your code here.
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

# to perform LDA, we must determine the minimum of n_features-1 or n_classes-1 as it will be the applied dimension.

# 1. determine the number of components for LDA
n_classes = len(y_train.unique())  
# print(n_classes) # there are two unique classes: 'unstable' and 'stable'

n_features = X_train_scaled.shape[1]  # number of features in X_train

# determining n_component for LDA
n_dimension = min(n_features - 1, n_classes - 1) 

# 2. initialize LDA
lda = LDA(n_components=n_dimension)

# 3. fit and transform the training data
X_train_lda = lda.fit_transform(X_train_scaled, y_train)

# 4. transform the test data
X_test_lda = lda.transform(X_test_scaled)

# create appropriate column names for the dataframe
columns = [f'LD{i+1}' for i in range(n_dimension)]

# first 5 rows of the LDA-transformed training data
print(pd.DataFrame(X_train_lda, columns=columns).head())

        LD1
0  1.825980
1  0.652440
2  2.911761
3 -0.364357
4  0.401231


### 8. Apply logistic regression to standartized X, PCA X and LDA X to predict Y, compare the result. Remember to split train and test sets. Set test sets to 20% of whole data. (10)

In [16]:
#write your code here
from sklearn.linear_model import LogisticRegression

# 1. initialize Logistic Regression
log_reg = LogisticRegression(random_state=42)

# 2. split the data into training and testing sets (20% for testing)
X_train_lr, X_test_lr, y_train_lr, y_test_lr = train_test_split(X_train_scaled, y_train, test_size=0.2, random_state=42)
X_train_pca_lr, X_test_pca_lr, y_train_pca_lr, y_test_pca_lr = train_test_split(X_train_pca, y_train, test_size=0.2, random_state=42)
X_train_lda_lr, X_test_lda_lr, y_train_lda_lr, y_test_lda_lr = train_test_split(X_train_lda, y_train, test_size=0.2, random_state=42)

# 3. fit and predict on standardized data
log_reg.fit(X_train_lr, y_train_lr)
y_pred_lr = log_reg.predict(X_test_lr)
accuracy_lr = accuracy_score(y_test_lr, y_pred_lr)

# 4. fit and predict on PCA reduced data
log_reg.fit(X_train_pca_lr, y_train_pca_lr)
y_pred_pca_lr = log_reg.predict(X_test_pca_lr)
accuracy_pca_lr = accuracy_score(y_test_pca_lr, y_pred_pca_lr)

# 5. fit and predict on LDA reduced data
log_reg.fit(X_train_lda_lr, y_train_lda_lr)
y_pred_lda_lr = log_reg.predict(X_test_lda_lr)
accuracy_lda_lr = accuracy_score(y_test_lda_lr, y_pred_lda_lr)

# X denotes the feature space
print(f"Logistic Regression Accuracy on Standardized X: {accuracy_lr}")
print(f"Logistic Regression Accuracy on PCA X: {accuracy_pca_lr}")
print(f"Logistic Regression Accuracy on LDA X: {accuracy_lda_lr}")

Logistic Regression Accuracy on Standardized X: 0.8130952380952381
Logistic Regression Accuracy on PCA X: 0.6617857142857143
Logistic Regression Accuracy on LDA X: 0.8121428571428572


### 9. Apply feature select to the STAB data.(10)

In [14]:
#Write your code here.
from sklearn.tree import DecisionTreeClassifier

# 1. Separate features and target
X = STAB.drop('stabf', axis=1)  # features (excluding 'stabf' and 'stab')
y = STAB['stabf']               # target variable 'stabf'

# 2. train a Decision Tree Classifier to get feature importances
model = DecisionTreeClassifier(random_state=42)
model.fit(X, y)

# 3. get feature importances
importances = model.feature_importances_

# 4. create a DataFrame for better visualization
feature_importances = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importances
})

# 5. sort features by importance
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

# 6. select top _n features (e.g., top 5)
top_n = 7
top_features = feature_importances.head(top_n)['Feature'].tolist()


print("Top 7 important features:")
print(top_features)


# transforming the dataset to keep only the top _n features
X_selected = X[top_features]

print(X_selected.head())

Top 7 important features:
['tau2', 'tau3', 'tau1', 'tau4', 'g1', 'g2', 'g4']
       tau2      tau3      tau1      tau4        g1        g2        g4
0  3.079885  8.381025  2.959060  9.780754  0.650456  0.859578  0.958034
1  4.902524  3.047541  9.304097  1.369357  0.413441  0.862414  0.781760
2  8.848428  3.046479  8.971707  1.214518  0.163041  0.766689  0.109853
3  7.669600  4.486641  0.716415  2.340563  0.446209  0.976744  0.362718
4  7.608772  4.943759  3.134112  9.857573  0.797110  0.455450  0.820923


### 10. Apply SVM to the edited STAB dataset, compare the result with the previous one.(Remember to split the dataset)(10)

In [15]:
#Write your code here.
from sklearn.svm import SVC


######## (a) preparing data for training ########

# (1) Separate features and target
X = STAB.drop('stabf', axis=1)  # all other features other than 'stabf'
y = STAB['stabf'] # target/response var, 'stabf'


# (2) split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.3, 
                                                    random_state=42)

X_train_selected, X_test_selected, y_train_selected, y_test_selected = train_test_split(X_selected,# (* X_selected was aquired from the feature selection made in the cell above!)
                                                                                        y, 
                                                                                        test_size=0.3, 
                                                                                        random_state=42)

####### (b) Train SVM Models ########

# train SVM on the original dataset
svm_original = SVC()
svm_original.fit(X_train, y_train)
y_pred_original = svm_original.predict(X_test)

# train SVM on the edited dataset (selected features)
svm_selected = SVC()
svm_selected.fit(X_train_selected, y_train_selected)
y_pred_selected = svm_selected.predict(X_test_selected)



####### (c) Evaluate Models #######

# Evaluate the SVM model on the original dataset
accuracy_original = accuracy_score(y_test, y_pred_original)
report_original = classification_report(y_test, y_pred_original)

# Evaluate the SVM model on the edited dataset (selected features)
accuracy_selected = accuracy_score(y_test_selected, y_pred_selected)
report_selected = classification_report(y_test_selected, y_pred_selected)


####### (d) Compare Results #######

print("SVM on Original Dataset:")
print(f"Accuracy: {accuracy_original}")
print(report_original)

print("\nSVM on Edited Dataset (Selected Features):")
print(f"Accuracy: {accuracy_selected}")
print(report_selected)


SVM on Original Dataset:
Accuracy: 0.9272222222222222
              precision    recall  f1-score   support

      stable       0.92      0.88      0.90      6522
    unstable       0.93      0.96      0.94     11478

    accuracy                           0.93     18000
   macro avg       0.93      0.92      0.92     18000
weighted avg       0.93      0.93      0.93     18000


SVM on Edited Dataset (Selected Features):
Accuracy: 0.885
              precision    recall  f1-score   support

      stable       0.86      0.82      0.84      6522
    unstable       0.90      0.92      0.91     11478

    accuracy                           0.89     18000
   macro avg       0.88      0.87      0.87     18000
weighted avg       0.88      0.89      0.88     18000



note to self(findings) <br>
I noticed that SVM applied to the selected feature set X yielded a lower score than when applied to the original dataset. This could be due to the Curse of Dimensionality, as reducing dataset components may impair the creation of optimal separating hyperplanes and lead to decreased model performance in high-dimensional data scenarios.